## Konwertowanie wszystkich plików JSON do DataFrame

In [1]:
import json
import glob
import os
import pandas as pd
import re

In [2]:
# funkcja do sortowania danych
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [3]:
# ścieżka do folderu w którym są pliki .tar
path_to_name='02_DANE/process/DANE_DLA_ANI/processed_bands_pow_ph/_coh/fold=6'

# tworzenie listy nazw z plików .tar
name_fold=[x.split('/')[-1].split('.')[0] for x in sorted(glob.glob('{}/*.tar'.format(path_to_name)),key=numericalSort)]

In [4]:
appended_data=[]
for x in name_fold:
    #podajemy ścieżkę do plików JSON
    path_to_json = '02_DANE/process/up_processed_bands_pow_ph/up_fold=6/{}'.format(x)
    json_files = [pos_json for pos_json in sorted(os.listdir(path_to_json),key=numericalSort) if pos_json.endswith('.json')]
    num=[x.split('.')[0] for x in json_files]
    df=[]
    for x,y in zip(json_files,num):
        json_file = json.load(open("{}/{}".format(path_to_json,x)))
        df_json = pd.DataFrame(json_file, index=[0])
        txt_file = open("{}/{}.cls".format(path_to_json,y), "r") #wczytujemy klasę z pliku .cls
        cls = txt_file.read()
        txt_file.close()
        df_json['class']=cls #dodajemy klasę 'chory/zdrowy' do df
        df_json['num']=y #dodajemy nr JSON do df
        df.append(df_json)
    result = pd.concat(df, ignore_index=True)
    appended_data.append(result)
appended_fold_6=pd.concat(appended_data, ignore_index=True)

In [5]:
# utworzenie klasy dla 'age'
appended_fold_6['age_int']=[int(appended_fold_6['age'][x]/10) for x in range(len(appended_fold_6['age']))]

In [6]:
appended_fold_6.head(5)

,patient_id,examination_id,age,gender,classification,description,institution_id,sampling_rate,signal_length_s,event_Eyes_Opened_length_s,event_Eyes_Closed_length_s,event_Stimulation_HyperventilationStart_length_s,event_Stimulation_PhotostimulationStart_length_s,event_type,class,num,age_int
0,{96566e47-7d7d-473a-9f81-70d33b57bbe8},20220509-154624-{bf77c62d-f6bb-4a44-8090-fa3cb...,50.750000,Male,normal,Zapis o niezbyt bogatej dość regularnej czynno...,OST,100.0,1349.99,24.0,180.0,300.0,168.0,None,0,4300,5
1,{ff5e27ac-9e77-4276-ade1-3f08814c36b8},20220405-111332-{8d3e1182-cf56-4121-b999-8b2ce...,24.000000,Female,other_patho,Badanie VIDEO-EEGczas trwania badania 1 h.Bada...,ZOZLO,100.0,3608.99,NaN,126.0,174.0,126.0,None,1,4301,2
2,{8d16e4b0-bf06-4930-aa3f-40ee68d1cd52},20220412-182433-{b4031717-e5b4-4798-8089-0955b...,52.060233,Male,epilepsy,"6,35adanie wykonano w czuwaniu. Czynność podst...",STG1,100.0,1079.99,84.0,258.0,210.0,156.0,None,1,4302,5
3,{ca3bd179-5886-4a84-8a7c-f17f9459b00a},20220512-174207-{33705535-1701-406d-93bc-8aa03...,53.750000,Female,epilepsy,Badanie wykonano w stanie czuwania Leki- Rejes...,CHE,100.0,1214.99,24.0,210.0,132.0,96.0,None,1,4303,5
4,{158e38e4-9246-4f8f-b34a-66914ba13b58},20220505-114852-{2ccddb13-4954-4e70-9ed8-504c2...,49.416667,Male,epilepsy,Czynność podstawową stanowi rytm alfa o dominu...,GAK,100.0,911.99,24.0,432.0,180.0,NaN,None,1,4304,4


In [7]:
# zapisujemy utworzony zbór danych
# appended_fold_6.to_csv('02_DANE/process/up_processed_bands_pow_ph/up_fold=6/fold_6.csv')